## load package

In [3]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# date
from datetime import datetime, timedelta

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# show all columns
from IPython.display import display
pd.options.display.max_columns = None

# memory manage
import gc

# logging
import logging 

# other
import tqdm as tqdm

# self define
sys.path.append('../../utils/')

In [4]:
logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler = logging.FileHandler('../log/extract_features.log')
handler.setLevel(logging.INFO)
formater = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formater)
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.info('This is a log info')

2018-11-03 13:47:06,992 - __main__ - INFO - This is a log info


In [5]:
# read files
lb_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2014.csv', sep = ';')
lb_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2015.csv', sep = ';')
lb_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2016.csv', sep = ';')
lb_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2017.csv', sep = ';')
lb_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2018.csv', sep = ';')
std_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2014.csv', sep = ';')
std_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2015.csv', sep = ';')
std_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2016.csv', sep = ';')
std_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2017.csv', sep = ';')
std_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2018.csv', sep = ';')
# cat
d1 = lb_wsp_2014.copy()
d2 = lb_wsp_2015.copy()
d3 = lb_wsp_2016.copy()
d4 = lb_wsp_2017.copy()
d5 = lb_wsp_2018.copy()

d6 = std_wsp_2014.copy()
d7 = std_wsp_2015.copy()
d8 = std_wsp_2016.copy()
d9 = std_wsp_2017.copy()
d10 = std_wsp_2018.copy()

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# for training we use data from 14 - 17 in lb_wsp
train1 = pd.concat([d1,d2,d3,d4,d5], 0)
train2 = pd.concat([d6,d7,d8,d9,d10], 0)
train1['Autohaus'] = 'leonberg'
train2['Autohaus'] = 'weil'
#train = pd.concat([train1, train2], 0)
#train = train1.copy()

In [7]:
# because there exists reused Auftragsnummer in different Autohaus station. So we add some sign to the Auftragsnummer
# in each Autohaus station
train1['Auftragsnummer'] = 'A' + train1['Auftragsnummer']
train2['Auftragsnummer'] = 'B' + train2['Auftragsnummer']
train = pd.concat([train1, train2], 0)

## extract Teile-Gruppe
An 4 . Stelle der Teile-Nr ist die Baugruppe 
- 1 - Motor
- 2 - Kraftstoff, Abgas, Kühlung
- 3 - Getrieb
- 4 - Vorderachse, Lenkung
- 5 - Hinterachse
- 6 - Räder
- 7 - Hebelwerk
- 8 - Karosserie
- 9 - Elektrik
- 0 - Zubehör, Infotainment, Sonstiges

#### remove the outlier in the Teile-Nr
- na wert
- Teile-Nr, which length smaller than 4
- doesn't contain number??
- doesn't contain adjoining number 

Question: The Gruppe-Nr can also in the 5. stelle<br>
#df[df['Teile-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', x[3])))]['Teile-Nr']<br>
the command show the Teile-Nr, which type are not number. We can see that many Teile-Nr doesn't contrain number; some Teile-Nr contain number, however there are not in the 4. stelle bu in the 5. oder 6. stelle. Therefore we suggest that the first coming adjoining nummer in the Teile-Nr may be the gruppe-Nr.<br>
We extract the gruppe-Nr according to this idea.<br>
This phase takes train data as input and output Dataframe df

In [8]:
# remove the outlier in the Teile-Nr
# remove the na wert
df = train[train['Teile-Nr'].isna().map(lambda x: not x)]
# remove the value short value
df = df[df['Teile-Nr'].map(lambda x: False if len(x) < 4 else True)]
# remove the value that doesn't contain number: 30593 of 593527 in train1(5%)
df = df[df['Teile-Nr'].map(lambda x: True if re.search('\d', x) else False)]
# remove the value that doesn't contrain adjoining number(min 2 number): 594 of 562934 in train1(0.1%)
df = df[df['Teile-Nr'].map(lambda x: True if re.search('\d\d', x) else False)]

In [9]:
# use findall instead of search, because, the df here should contain adjoining number, otherwise it's wrong
df['Gruppe-Nr'] = df['Teile-Nr'].map(lambda x: re.findall('\d\d', x)[0][0])

In [10]:
df['Gruppe-Nr'].value_counts()

0    930093
1    281620
9    241658
3     46656
2     37419
6     23866
8     21086
5     19447
4     14385
7      6041
Name: Gruppe-Nr, dtype: int64

In [11]:
# clean the memory
del train1, train2, lb_wsp_2014, lb_wsp_2015, lb_wsp_2016, lb_wsp_2017, lb_wsp_2018
del std_wsp_2014, std_wsp_2015, std_wsp_2016, std_wsp_2017, std_wsp_2018
del d1, d2, d3, d4, d5, d6, d7, d8, d9, d10
gc.collect()

91

In [12]:
#

## remove the confused data
- For the same Auftragsnummer, exists more than one value in the other attribute.
- wierd Auftragsnummer: 107K, 77KW
- Fahrgestellnummer is Na
- Fahrgestellnummer that only contrain number.
- In Auftragsdatum, the day of the datum is larger than 31
- Wierd Auftragsdatum, these datum is shorter than 10 Ziffer.
- Wierd Auftragsdatum, these datum is smaller than 2013

In [13]:
# not only see that data with gruppe-nr == 9, 241658 items
# the target of next coming can also for the other thing, not the eletric
# df = df[df['Gruppe-Nr'] == '9']

In [14]:
# choose the features, which may be essentail to the model
# We will take care of the 'AW-Nr' and 'KM-Stand' in the other place.
# there are 98256 unique items in cf, more than 90423 items in af, which means they may create NA value after merging
#cf = train[['Auftragsnummer', 'AW-Nr', 'Markencode', 'Lagerortcode', 'Adressanredecode', 'Motorcode', 'Fahrzeugmodellnummer', 'Modell', 'Typ',
#        'Getriebecode', 'Gewicht', 'Leistung (KW)', 'Fahrgestellnummer']]
cf = df.copy()

In [15]:
# drop duplicate
cf = cf.drop_duplicates()

In [16]:
# find out the confused data. For the same Auftragsnummer, exists more than one value in the other attribute.
# here is AWSAU310019, BWSAU386471, BWSAU435051
cf = cf.drop(cf[cf['Auftragsnummer'] == 'AWSAU310019'].index, axis= 0) # 8 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU386471'].index, axis= 0) # 3 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU435051'].index, axis= 0) # 2 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU271939'].index, axis= 0) # ? items

In [17]:
# remove wierd auftragsnummer
cf = cf.drop(cf[cf['Auftragsnummer'] == '103K'].index) #
cf = cf.drop(cf[cf['Auftragsnummer'] == '77KW'].index) #

In [18]:
# remove na value in Fahrgestellnummer
cf = cf.drop(cf[cf['Fahrgestellnummer'].isna()].index, axis = 0) #

In [19]:
# remove the Fahrgestellnumer that only contain number, not tested!!!!
cf = cf.drop(cf[cf['Fahrgestellnummer'].map(lambda x: False if re.search('[a-zA-Z]', x) else True)].index, axis = 0)

In [20]:
# remove the items, which day of the Auftragsdatum lareger than 31, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x: int(x[0:2])) > 31].index, axis = 0)

In [21]:
# remove the items, which length of the Auftragsdatum shorter than 10, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x:True if len(x) < 10 else False)].index, axis = 0)

In [22]:
# remove the items, which year of the Auftragsdatum smaller than 2013, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x: int(x[6:]) < 2013)].index, axis = 0)

In [23]:
#len(cf)

In [24]:
df = cf
del cf
gc.collect()

168

In [ ]:
#check_to_1(tmp, 'Auftragsnummer', 'Markencode')

In [ ]:
#check_to_1(tmp, 'Auftragsnummer', 'Lagerortcode')

In [18]:
#check_to_1(tmp, 'Auftragsnummer', 'Adressanredecode')

245023
21
BWSAU435051	['Herr', 'Firma']


In [19]:
#check_to_1(tmp, 'Auftragsnummer', 'Motorcode')

245023
1267
AWSAU310019	['DDYA', 'AKQ']
BWSAU435051	['CFHC', 'CRLB']


In [20]:
#check_to_1(tmp, 'Auftragsnummer', 'Fahrzeugmodellnummer')

245023
7134
AWSAU310019	['BV527Z', '6X10Z4']
BWSAU386471	['NJ33E4', '']
BWSAU435051	['1T33A3', '3V5455']


In [21]:
#check_to_1(tmp, 'Auftragsnummer', 'Modell')

245023
10538
AWSAU310019	['Golf Variant Trendline', 'VW Lupo > 99']
BWSAU386471	['Fabia Ambition 1,0l MPI 55 kW', '']
BWSAU435051	['Touran Comfortline 2,0 l TDI', 'Superb Combi Style TDI 110 kW']


In [22]:
#check_to_1(tmp, 'Auftragsnummer', 'Typ')

245023
557
AWSAU310019	['X0A', '']
BWSAU435051	['X0A', '']


In [23]:
#check_to_1(tmp, 'Auftragsnummer', 'Getriebecode')

245023
2799
AWSAU310019	['GKB', 'DKF']
BWSAU435051	['PPY', 'QFZ']


In [24]:
#check_to_1(tmp, 'Auftragsnummer', 'Gewicht')

245023
53
BWSAU386471	['0,00', '']


In [25]:
#check_to_1(tmp, 'Auftragsnummer', 'Leistung (KW)')

245023
143
AWSAU310019	['85.0', '0.0']
BWSAU386471	['0.0', '']
BWSAU435051	['103.0', '0.0']


In [26]:
#check_to_1(tmp, 'Auftragsnummer', 'Fahrgestellnummer')

245023
70747
AWSAU310019	['WVWZZZAUZJP531831', 'WVWZZZ6XZXW035409']
BWSAU386471	['TMBEB6NJ7JZ032235', '']
BWSAU435051	['WVGZZZ1TZEW001359', 'TMBJH7NP5J7507173']


In [100]:
#check_to_1(train, 'Auftragsnummer', 'KM-Stand')

292528
124554


In [102]:
#check_to_1(train, 'Auftragsnummer', 'Erstzulassungsdatum')

292528
6613
AWSAU310019	['27.09.2017', '01.03.1999']
BWSAU271939	['14.04.2009', '22.06.2012']
BWSAU386471	['18.09.2017', '']
BWSAU435051	['03.06.2013', '21.07.2017']


## Get Auftrag table for Gruppe-Nr

In [25]:
gn = df.copy()

In [26]:
gn = toAuftragTable(gn, 'Gruppe-Nr', 'Auftragsnummer') # number: 245018

Falls Null date exist, drop these dates directly


In [27]:
len(gn)

244077

## extract feature: total AW-Nr
extract the feature total AW and combine aw with gruppe-nr auftragstable to form a new Dataframe agn. <br>
combine the attribute in df to the agn

In [28]:
aw = train.copy() # can't use df here, because df delete all the na value in Teile-Nr

In [29]:
# remove the items, that contain ziffer in the AW-Nr, set data type to int
aw['AW-Nr'][aw['AW-Nr'].isnull()] = 0 # set null value to 0
aw['AW-Nr'][aw['AW-Nr'] == '99999999'] = 0 # set 99999999 to 0
aw['AW-Nr'] = aw['AW-Nr'].map(lambda x: re.sub(' ', '', str(x)))# remove space between the number
del_index = aw[aw['AW-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', str(x))))].index # remove fake data
aw = aw.drop(del_index) 
aw['AW-Nr'] = aw['AW-Nr'].astype('float')

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# to auftrag table, get the feature: total AW-Nr for each auftrag
tmp = aw[['AW-Nr', 'Auftragsnummer']]
ag = tmp.groupby('Auftragsnummer', as_index = False)
out = []
for name, gruppe in ag:
    out.append(pd.DataFrame({'Auftragsnummer': [gruppe['Auftragsnummer'].iloc[0]], 'total AW': [gruppe['AW-Nr'].sum()]}))
aw = pd.concat(out).reset_index()[['Auftragsnummer', 'total AW']] # 缺分段
# drop the wierd Auftragsnummer
aw = aw.drop(aw[aw['Auftragsnummer'] == '103K'].index)
aw = aw.drop(aw[aw['Auftragsnummer'] == '77KW'].index)

In [31]:
# the lengths of aw and gn are different, because in some auftrag, the customer do not buy anything. 
# These part of auftrags are not included in gn, but gn is not a subset of aw. 
# because some AW-Nr in aw are wrong, we have deleted these items in aw. 
# Therefore, if we want to merge aw and gn, we have to use the method inner
agn = pd.merge(aw, gn, how = 'inner', on = 'Auftragsnummer')

## extract feature: num_act
not tested jet<br>
make sure that, the length of the output, shoube be the same as the length of agn

In [32]:
aw = train.copy()
# remove the items, that contain ziffer in the AW-Nr, set data type to int
aw['AW-Nr'][aw['AW-Nr'].isnull()] = 0 # set null value to 0
aw['AW-Nr'][aw['AW-Nr'] == '99999999'] = 0 # set 99999999 to 0
aw['AW-Nr'] = aw['AW-Nr'].map(lambda x: re.sub(' ', '', str(x)))# remove space between the number
del_index = aw[aw['AW-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', str(x))))].index # remove fake data
aw = aw.drop(del_index) 
aw['AW-Nr'] = aw['AW-Nr'].astype('float')
# to auftrag table, get the feature: total AW-Nr for each auftrag
tmp = aw[['AW-Nr', 'Auftragsnummer']]
ag = tmp.groupby('Auftragsnummer', as_index = False)
out = []
for name, gruppe in ag:
    out.append(pd.DataFrame({'Auftragsnummer': [gruppe['Auftragsnummer'].iloc[0]], 'num_act': [len(gruppe)]}))
aw = pd.concat(out).reset_index()[['Auftragsnummer', 'num_act']] # 缺分段
# drop the wierd Auftragsnummer
aw = aw.drop(aw[aw['Auftragsnummer'] == '103K'].index)
aw = aw.drop(aw[aw['Auftragsnummer'] == '77KW'].index)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [33]:
#agn = pd.merge(aw, agn, how = 'inner', on = 'Auftragsnummer')
agn = pd.merge(aw, agn, how = 'inner', on = 'Auftragsnummer')

In [34]:
agn.head()

,Auftragsnummer,num_act,total AW,Gruppe-Nr
0,AWSAU195981,31,1.526062e+09,3;0
1,AWSAU198256,31,6.398138e+08,0;9;1
2,AWSAU200129,14,3.832960e+08,9;6;1;0
3,AWSAU200223,60,1.841819e+09,0;6;5;3;9;1
4,AWSAU200258,23,5.367113e+08,0;9;1


In [35]:
#train[train['Auftragsnummer'] == 'AWSAU195981']

## extract feature: num_teile

In [36]:
agn['num_teile'] = agn['Gruppe-Nr'].map(lambda x: len(x.split(';')))

In [37]:
agn[['total AW', 'num_act']].corr() 
# why the correlation is so small??

,total AW,num_act
total AW,1.000000,0.584288
num_act,0.584288,1.000000


In [38]:
del aw, gn
gc.collect()

112

## combine agn with df

In [39]:
tmp = df[['Auftragsnummer', 'KM-Stand','Markencode', 'Lagerortcode', 'Auftragsdatum', 'Adressanredecode', 'Motorcode', 'Fahrzeugmodellnummer', 'Modell', 'Typ', 
          'Getriebecode', 'Gewicht', 'Leistung (KW)', 'Fahrgestellnummer', 'Erstzulassungsdatum']]
tmp = tmp.drop_duplicates()
tmp = pd.merge(agn, tmp, how = 'inner', on = 'Auftragsnummer') # 244608

In [40]:
df = tmp
del tmp
gc.collect()

41

In [41]:
#TODO
# calculate the length of aw and original df
# i suggest that, because i delete all the na value in Teile-Nr, the number of items in original df is smaller
# than the number in aw.
# the result is, after mergin, there are many na value in some attribute, 
# i want to see the number of distance.
# or may be i can merge the data in the other direction: 'right'
# but mit direction right, there exist also 600 null value but in total AN, the reason may be
# that i delete some fake data in aw.
# the problem here is, should i maintain the na value in total AW, other just remove all of them?

## extract features: month, day

In [42]:
md = df.copy()
# remove the outlier in the data
#fake = md[md['Auftragsdatum'].map(lambda x:True if re.search('[a-zA-Z]', x) else False)]
#md = md.drop(fake.index, axis=0)

In [43]:
md['year'] = md['Auftragsdatum'].map(lambda x: x[6:])
md['month'] = md['Auftragsdatum'].map(lambda x: x[3:5])
md['day'] = md['Auftragsdatum'].map(lambda x: x[0:2])

In [44]:
df = md
del md
gc.collect()

28

In [45]:
df['num_teile'].value_counts()

1     99287
3     66428
2     50047
4     20344
5      5373
6      1446
7       494
8       185
9        53
10       10
Name: num_teile, dtype: int64

## extract feature: age
not tested jet!!!<br>
age_auto = auftragsdatum - Erstzulassungsdatum<br>
age_autohaus = auftragsdatum - 2013

In [46]:
#tmp = train[['Auftragsdatum', 'Erstzulassungsdatum']]
tmp = df.copy()

In [47]:
tmp['Auftragsdatum'] = pd.to_datetime(tmp['Auftragsdatum'], dayfirst=True)
tmp['Erstzulassungsdatum'] = pd.to_datetime(tmp['Erstzulassungsdatum'], dayfirst=True)
tmp['age_auto'] = tmp['Auftragsdatum'] - tmp['Erstzulassungsdatum']

In [48]:
tmp['age_autohaus'] = tmp['Auftragsdatum'].map(lambda x: x.year) - 2013

In [49]:
df = tmp
del tmp
gc.collect()

14

In [50]:
df.head(10)

,Auftragsnummer,num_act,total AW,Gruppe-Nr,num_teile,KM-Stand,Markencode,Lagerortcode,Auftragsdatum,Adressanredecode,Motorcode,Fahrzeugmodellnummer,Modell,Typ,Getriebecode,Gewicht,Leistung (KW),Fahrgestellnummer,Erstzulassungsdatum,year,month,day,age_auto,age_autohaus
0,AWSAU195981,31,1.526062e+09,3;0,2,"38110,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-13,Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627,2013-05-17,2014,01,13,241 days,1
1,AWSAU198256,31,6.398138e+08,0;9;1,3,"59660,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-16,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,2012-07-27,2014,01,16,538 days,1
2,AWSAU200129,14,3.832960e+08,9;6;1;0,4,"227777,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-09,Herr,CBAB,5K143M,"Golf Highline 2,0 l",X0A,LQV,"0,00",0.0,WVWZZZ1KZAW187314,2009-11-25,2014,01,09,1506 days,1
3,AWSAU200223,60,1.841819e+09,0;6;5;3;9;1,6,"65174,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-07,Herr,AXR,1J5134,VW Golf IV > 98 / Variant > 99,NaN,EUH,"0,00",0.0,WVWZZZ1JZ2W511716,2002-02-15,2014,01,07,4344 days,1
4,AWSAU200258,23,5.367113e+08,0;9;1,3,"212110,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Firma,CAGA,8K50QC,A4 Avant 2.0 TDI Attraction,X0A,LLN,"0,00",105.0,WAUZZZ8K5AA164327,2010-05-06,2014,01,02,1337 days,1
5,AWSAU200259,9,1.862719e+08,0;9,2,"91895,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,ALT,8E20B4,A4 Limousine 2.0 R4 96 M5S,NaN,GBM,"0,00",0.0,WAUZZZ8E13A363100,2003-04-24,2014,01,02,3906 days,1
6,AWSAU200260,11,3.962096e+08,0,1,"122325,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,CAYC,8P1AE4,Audi A3 Attraction 3-Türer 1.6,X0A,LUB,"0,00",0.0,WAUZZZ8PXAA157479,2010-04-29,2014,01,02,1344 days,1
7,AWSAU200263,36,6.978901e+08,0;3;9;1,4,"112386,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,ASB,4F20LL,Audi A6 Limousine 3.0 TDI (DPF,X0A,KGX,"0,00",0.0,WAUZZZ4F28N185258,2008-10-04,2014,01,02,1916 days,1
8,AWSAU200264,12,1.656055e+08,0;8;3,3,"42783,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,CLAB,4G50NH,A6 Avant 3.0 TDI,X0A,NKP,"0,00",150.0,WAUZZZ4G4CN101295,2012-02-21,2014,01,02,681 days,1
9,AWSAU200269,8,1.641055e+08,0;3,2,"180999,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,BPP,8ED0YH,Audi A4 Avant 2.7 TDI(DPF),X0A,JBB,"0,00",0.0,WAUZZZ8E07A224307,2007-03-29,2014,01,02,2471 days,1


## transfer the type of km stand to float

In [51]:
df['KM-Stand'] = df['KM-Stand'].map(lambda x: float(re.sub(',', '.', x)))

## extract feature: KM age

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243667 entries, 0 to 243666
Data columns (total 24 columns):
Auftragsnummer          243667 non-null object
num_act                 243667 non-null int64
total AW                243667 non-null float64
Gruppe-Nr               243667 non-null object
num_teile               243667 non-null int64
KM-Stand                243667 non-null float64
Markencode              243468 non-null object
Lagerortcode            243667 non-null object
Auftragsdatum           243667 non-null datetime64[ns]
Adressanredecode        241586 non-null object
Motorcode               235550 non-null object
Fahrzeugmodellnummer    243477 non-null object
Modell                  242305 non-null object
Typ                     162462 non-null object
Getriebecode            234752 non-null object
Gewicht                 243461 non-null object
Leistung (KW)           243461 non-null float64
Fahrgestellnummer       243667 non-null object
Erstzulassungsdatum     243461 non

## extract feature: time relative
- times in group
- time distance since last come
- time distance to next come
- km stand distance since last come(correct the data first)
- km stand distance to next come
- is km stand to/from next/last come larger than threshold
- frequent in each year, in total
- (not jet)weather come in X month again
- time shift for num_act, total aw, Gruppe-Nr, num_teile
- repair age in year or in month
- repair frequent in last X month
- trend of frequent in the past
- mean, max, min, quater repaire km distance
- mean, max, min, quater repaire time distance

We sort the dataframe with ['Fahrgestellnummer', 'Auftragsdatum'], and calculate the datum distance betwenn the nearby items.<br>
One thing to be clear is that, the first distance for each Fahrgestellnummer is wrong, because we calculate it with the its Auftragsdatum and the Auftragsdatum of the other Fahrgestellnummer.<br>

In [53]:
td = df.copy()

In [54]:
td = td[['Fahrgestellnummer', 'Auftragsdatum', 'KM-Stand','num_act', 'total AW', 'num_teile', 'year']]
#1C8FYN8O63T585854 8

In [55]:
# get sample, to test the code
for name, group in td.groupby('Fahrgestellnummer', as_index = False):
    if name == '1C8FYN8O63T585854':
        tmp = group.reindex()
        print(tmp.columns)
        print(tmp.index)
        # time distance since/to last/next come
     #create feature: time distance from last coming

Index(['Fahrgestellnummer', 'Auftragsdatum', 'KM-Stand', 'num_act', 'total AW',
       'num_teile', 'year'],
      dtype='object')
Int64Index([103323, 108984, 112403, 114316, 125322, 159899, 171613, 193678], dtype='int64')


In [190]:
# code to extract new features, remmenber to remove counter
out = []
counter = 0
for name, group in td.groupby('Fahrgestellnummer', as_index = False):
    counter = counter + 1
    if counter > 1000:
        break
    #if name == '1C8FYN8O63T585854':
    #    tmp = group.reindex()
    #    print(tmp.columns)
    #    print(tmp.index)
    #    # time distance since/to last/next come
    # create feature: time distance from last coming
    tmp = group
    if len(tmp) > 1:
        tmp['times'] = len(tmp)
        tmp = tmp.sort_values(by = 'Auftragsdatum').reset_index().iloc[:,1:] # not tested
        # create feature: time distance from last coming
        time_shift_forward = pd.concat([pd.Series(datetime(2010, 1, 1)), tmp['Auftragsdatum'][:-1]])
        time_shift_forward.index = tmp.index
        tmp['time_distance_from_last'] = tmp['Auftragsdatum'] - time_shift_forward
        tmp['time_distance_from_last'] = tmp['time_distance_from_last'].clip(lower = timedelta(0), upper = timedelta(365)) # clip
        # create feature: mean time distance in group, should i clip the data first????
        tmp['mean_time_distance_in_group'] = tmp['time_distance_from_last'].mean()
        # create feature: max time distance in group
        tmp['max_time_distance_in_group'] = tmp['time_distance_from_last'].max()
        # create feature: min time distance in group
        tmp['min_time_distance_in_group'] = tmp['time_distance_from_last'].min()
        # create feature: time distance to next coming
        time_distance_to_next = pd.concat([tmp['time_distance_from_last'][1:], pd.Series(np.NaN)])
        time_distance_to_next.index = tmp.index
        tmp['time_distance_to_next'] = time_distance_to_next
        # reset the first value and last value
        tmp.iloc[0, tmp.columns.get_loc('time_distance_from_last')] = np.NaN
        tmp.iloc[-1, tmp.columns.get_loc('time_distance_to_next')] = np.NaN
        # create feature: km stand distance from last coming
        km_shift_forward = pd.concat([pd.Series(0.0), tmp['KM-Stand'][:-1]])
        km_shift_forward.index = tmp.index
        tmp['km_distance_from_last'] = tmp['KM-Stand'] -km_shift_forward
        tmp['km_distance_from_last'] = tmp['km_distance_from_last'].abs() # correct the negative value in a stupid way!!!
        tmp['km_distance_from_last'][tmp['km_distance_from_last'] > 10000] = 10000 # clip
        # create feature: mean km stand distance, not test
        tmp['mean_km_distance_in_group'] = tmp['km_distance_from_last'].mean() 
        # create feature: max km stand distance, not test
        tmp['max_km_distance_in_group'] = tmp['km_distance_from_last'].max()
        # create feature: min km stand distance, not test
        tmp['min_km_distance_in_group'] = tmp['km_distance_from_last'].min()
        # create feature: km stand distance to next coming
        km_distance_to_next = pd.concat([tmp['km_distance_from_last'][1:], pd.Series(0.0)])
        km_distance_to_next.index = tmp.index
        tmp['km_distance_to_next'] = km_distance_to_next
        # reset the first value and last value
        tmp.iloc[0, tmp.columns.get_loc('km_distance_from_last')] = np.NaN
        tmp.iloc[-1, tmp.columns.get_loc('km_distance_to_next')] = np.NaN
        # create feature: frequent in each year
        year_candidates = ['2014', '2015', '2016', '2017', '2018']
        for year in year_candidates:
            tmp['frequent_in_'+year] = len(tmp[tmp['year'] == year])
        # create feature: time shift for num_act
        num_act_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_act'][:-1]])
        num_act_shift_forward.index = tmp.index
        tmp['num_act_shift1'] = num_act_shift_forward
        # create feature: time shift for num_teile
        num_teile_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_teile'][:-1]])
        num_teile_shift_forward.index = tmp.index
        tmp['num_teile_shift1'] = num_teile_shift_forward
        # create feature: time shift for total aw
        total_aw_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['total AW'][:-1]])
        total_aw_shift_forward.index = tmp.index
        tmp['total_aw_shift1'] = total_aw_shift_forward
        if len(tmp) > 2:
            num_act_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_act'][:-2]])
            num_act_shift_forward2.index = tmp.index
            tmp['num_act_shift2'] = num_act_shift_forward2
            num_teile_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_teile'][:-2]])
            num_teile_shift_forward2.index = tmp.index
            tmp['num_teile_shift2'] = num_teile_shift_forward2
            total_aw_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['total AW'][:-2]])
            total_aw_shift_forward2.index = tmp.index
            tmp['total_aw_shift2'] = total_aw_shift_forward2
        else:
            tmp['num_act_shift2'] = np.NaN
            tmp['num_teile_shift2'] = np.NaN
            tmp['total_aw_shift2'] = np.NaN
        # create feature: repair age in year or in month
        tmp['first_date'] = tmp.iloc[0, tmp.columns.get_loc('Auftragsdatum')]
        tmp['repair_age'] = tmp['Auftragsdatum'] - tmp['first_date']
        tmp = tmp.drop('first_date', axis=1)
        # create feature: repair frequent in last X month
        time_distance_shift1 = pd.concat([pd.Series(np.NaN), tmp['time_distance_from_last'][:-1]])
        time_distance_shift1.index = tmp.index
        tmp['time_distance_shift1'] = time_distance_shift1
        time_distance_shift2 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift1'][:-1]])
        time_distance_shift2.index = tmp.index
        tmp['time_distance_shift2'] = time_distance_shift2
        time_distance_shift3 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift2'][:-1]])
        time_distance_shift3.index = tmp.index
        tmp['time_distance_shift3'] = time_distance_shift3
        tmp['time_distance_from_last2'] = tmp['time_distance_from_last'] + tmp['time_distance_shift1']
        tmp['time_distance_from_last3'] = tmp['time_distance_from_last2'] + tmp['time_distance_shift2']
        tmp['time_distance_from_last4'] = tmp['time_distance_from_last3'] + tmp['time_distance_shift3']
        def get_frequent(x, d = 30):
            if x['time_distance_from_last4'] < timedelta(days = d):
                return 4
            elif x['time_distance_from_last3'] < timedelta(days = d):
                return 3
            elif x['time_distance_from_last2'] < timedelta(days = d):
                return 2
            elif x['time_distance_from_last'] < timedelta(days = d):
                return 1
            else:
                return 0
        tmp['frequent_in_last_1_month'] = tmp.apply(get_frequent, axis=1, args = [1*30])
        tmp['frequent_in_last_2_month'] = tmp.apply(get_frequent, axis=1, args = [2*30])
        tmp['frequent_in_last_3_month'] = tmp.apply(get_frequent, axis=1, args = [3*30])
        tmp['frequent_in_last_4_month'] = tmp.apply(get_frequent, axis=1, args = [4*30])
        tmp['frequent_in_last_5_month'] = tmp.apply(get_frequent, axis=1, args = [5*30])
        tmp['frequent_in_last_6_month'] = tmp.apply(get_frequent, axis=1, args = [6*30])
        # create feature: weather come in last X month
        tmp['weather_come_in_last_1_month'] = tmp['time_distance_from_last'] < timedelta(days = 1*30)
        tmp['weather_come_in_last_2_month'] = tmp['time_distance_from_last'] < timedelta(days = 2*30)
        tmp['weather_come_in_last_3_month'] = tmp['time_distance_from_last'] < timedelta(days = 3*30)
        tmp['weather_come_in_last_4_month'] = tmp['time_distance_from_last'] < timedelta(days = 4*30)
        tmp['weather_come_in_last_5_month'] = tmp['time_distance_from_last'] < timedelta(days = 5*30)
        tmp['weather_come_in_last_6_month'] = tmp['time_distance_from_last'] < timedelta(days = 6*30)
        # create feature: trend of frequent in the past
        def get_trend(x):
            xs = x[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].tolist()
            xs = [x.days for x in xs if not pd.isnull(x)]
            if len(xs) < 2:
                return 0
            ys = range(1, len(xs) + 1)
            xm = sum(xs)/len(xs)
            ym = sum(ys)/len(ys)
            xy = [x*y for x, y in zip(xs, ys)]
            xx = [x*y for x, y in zip(xs, xs)]
            up = sum(xy) - len(xs)*xm*ym
            down = sum(xx) - len(xs)*xm*xm
            if down == 0:
                return 0
            return up*1.0/down
        tmp['trend_of_frequent_in_the_past'] = tmp.apply(get_trend, axis=1)
        # mean km distance:
    else:
        # set features to NaN
        tmp['times'] = 1
        tmp['time_distance_from_last'] = pd.NaT
        tmp['time_distance_to_next'] = pd.NaT
        tmp['km_distance_from_last'] = pd.NaT
        tmp['km_distance_to_next'] = pd.NaT
        tmp['frequent_in_2014'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2014' else 0
        tmp['frequent_in_2015'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2015' else 0
        tmp['frequent_in_2016'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2016' else 0
        tmp['frequent_in_2017'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2017' else 0
        tmp['frequent_in_2018'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2018' else 0
        tmp['num_act_shift1'] = np.NaN
        tmp['num_teile_shift1'] = np.NaN
        tmp['total_aw_shift1'] = np.NaN
        tmp['num_act_shift2'] = np.NaN
        tmp['num_teile_shift2'] = np.NaN
        tmp['total_aw_shift2'] = np.NaN
        tmp['time_distance_shift1'] = pd.NaT
        tmp['time_distance_shift2'] = pd.NaT
        tmp['time_distance_shift3'] = pd.NaT
        tmp['time_distance_from_last2'] = pd.NaT
        tmp['time_distance_from_last3'] = pd.NaT
        tmp['time_distance_from_last4'] = pd.NaT
        tmp['weather_come_in_last_1_month'] = False
        tmp['weather_come_in_last_2_month'] = False
        tmp['weather_come_in_last_3_month'] = False
        tmp['weather_come_in_last_4_month'] = False
        tmp['weather_come_in_last_5_month'] = False
        tmp['weather_come_in_last_6_month'] = False
        tmp['trend_of_frequent_in_the_past'] = 0
    out.append(tmp)

# i don't known why the warning appear??? try to figure it out, it can save lots of time.

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [191]:
df3 = pd.concat(out).reset_index().iloc[:, 1:]

In [192]:
df3

,Auftragsdatum,Fahrgestellnummer,KM-Stand,frequent_in_2014,frequent_in_2015,frequent_in_2016,frequent_in_2017,frequent_in_2018,frequent_in_last_1_month,frequent_in_last_2_month,frequent_in_last_3_month,frequent_in_last_4_month,frequent_in_last_5_month,frequent_in_last_6_month,km_distance_from_last,km_distance_to_next,max_km_distance_in_group,max_time_distance_in_group,mean_km_distance_in_group,mean_time_distance_in_group,min_km_distance_in_group,min_time_distance_in_group,num_act,num_act_shift1,num_act_shift2,num_teile,num_teile_shift1,num_teile_shift2,repair_age,time_distance_from_last,time_distance_from_last2,time_distance_from_last3,time_distance_from_last4,time_distance_shift1,time_distance_shift2,time_distance_shift3,time_distance_to_next,times,total AW,total_aw_shift1,total_aw_shift2,trend_of_frequent_in_the_past,weather_come_in_last_1_month,weather_come_in_last_2_month,weather_come_in_last_3_month,weather_come_in_last_4_month,weather_come_in_last_5_month,weather_come_in_last_6_month,year
0,2014-05-20,------AU-00201251,266265.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,16,NaN,NaN,2,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014
1,2015-08-04,159247535XXXXXXXX,87944.0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,42,NaN,NaN,5,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,6.920609e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2015
2,2014-07-26,1C3ANE9L24X016248,0.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,3,NaN,NaN,1,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014
3,2014-10-20,1C8FYN8O63T585854,107950.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2192,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,10,NaN,NaN,2,NaN,NaN,0 days,NaT,NaT,NaT,NaT,NaN,NaN,NaN,53 days 00:00:00,8,2.506427e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2014
4,2014-12-12,1C8FYN8O63T585854,110142.0,2,3,2,1,0,0.0,1.0,1.0,1.0,1.0,1.0,2192,636,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,8,10.0,NaN,2,2.0,NaN,53 days,53 days 00:00:00,NaT,NaT,NaT,NaT,NaN,NaN,52 days 00:00:00,8,7.282570e+07,2.506427e+08,NaN,0.000000,False,True,True,True,True,True,2014
5,2015-02-02,1C8FYN8O63T585854,110778.0,2,3,2,1,0,0.0,1.0,1.0,2.0,2.0,2.0,636,587,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,14,8.0,10.0,6,2.0,2.0,105 days,52 days 00:00:00,105 days 00:00:00,NaT,NaT,53 days 00:00:00,NaT,NaN,23 days 00:00:00,8,5.437718e+07,7.282570e+07,2.506427e+08,1.000000,False,True,True,True,True,True,2015
6,2015-02-25,1C8FYN8O63T585854,111365.0,2,3,2,1,0,1.0,1.0,2.0,2.0,3.0,3.0,587,1565,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,4,14.0,8.0,2,6.0,2.0,128 days,23 days 00:00:00,75 days 00:00:00,128 days 00:00:00,NaT,52 days 00:00:00,53 days 00:00:00,NaT,117 days 00:00:00,8,8.299390e+07,5.437718e+07,7.282570e+07,0.051665,True,True,True,True,True,True,2015
7,2015-06-22,1C8FYN8O63T585854,112930.0,2,3,2,1,0,0.0,0.0,0.0,1.0,2.0,2.0,1565,8010,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,5,4.0,14.0,2,2.0,6.0,245 days,117 days 00:00:00,140 days 00:00:00,192 days 00:00:00,245 days 00:00:00,23 days 00:00:00,52 days 00:00:00,53 days 00:00:00,354 days 00:00:00,8,1.740418e+08,8.299390e+07,5.437718e+07,-0.017250,False,False,False,True,True,True,2015
8,2016-06-10,1C8FYN8O63T585854,120940.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,8010,1138,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,10,5.0,4.0,2,2.0,2.0,599 days,354 days 00:00:00,471 days 00:00:00,494 days 00:00:00,546 days 00:00:00,117 days 00:00:00,23 days 00:00:00,52 days 00:00:00,123 days 00:00:00,8,1.449721e+08,1.740418e+08,8.299390e+07,-0.007385,False,False,False,False,False,False,2016
9,2016-10-11,1C8FYN8O63T585854,122078.0,2,3,2,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1138,4162,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,15,10.0,5.0,3,2.

In [193]:
# test code
# create feature: time distance from last coming
time_shift_forward = pd.concat([pd.Series(datetime(2010, 1, 1)), tmp['Auftragsdatum'][:-1]])
time_shift_forward.index = tmp.index
tmp['time_distance_from_last'] = tmp['Auftragsdatum'] - time_shift_forward
# create feature: time distance to next coming
time_distance_to_next = pd.concat([tmp['time_distance_from_last'][1:], pd.Series(np.NaN)])
time_distance_to_next.index = tmp.index
tmp['time_distance_to_next'] = time_distance_to_next
# reset the first value and last value
tmp.iloc[0, tmp.columns.get_loc('time_distance_from_last')] = np.NaN
tmp.iloc[-1, tmp.columns.get_loc('time_distance_to_next')] = np.NaN
# create feature: km stand distance from last coming
km_shift_forward = pd.concat([pd.Series(0.0), tmp['KM-Stand'][:-1]])
km_shift_forward.index = tmp.index
tmp['km_distance_from_last'] = tmp['KM-Stand'] -km_shift_forward
# create feature: km stand distance to next coming
km_distance_to_next = pd.concat([tmp['km_distance_from_last'][1:], pd.Series(0.0)])
km_distance_to_next.index = tmp.index
tmp['km_distance_to_next'] = km_distance_to_next
# reset the first value and last value
tmp.iloc[0, tmp.columns.get_loc('km_distance_from_last')] = np.NaN
tmp.iloc[-1, tmp.columns.get_loc('km_distance_to_next')] = np.NaN
# create feature: frequent in each year
year_candidates = ['2014', '2015', '2016', '2017', '2018']
for year in year_candidates:
    tmp['frequent_in_'+year] = len(tmp[tmp['year'] == year])
# create feature: time shift for num_act
num_act_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_act'][:-1]])
num_act_shift_forward.index = tmp.index
tmp['num_act_shift1'] = num_act_shift_forward
# create feature: time shift for num_teile
num_teile_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_teile'][:-1]])
num_teile_shift_forward.index = tmp.index
tmp['num_teile_shift1'] = num_teile_shift_forward
# create feature: time shift for total aw
total_aw_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['total AW'][:-1]])
total_aw_shift_forward.index = tmp.index
tmp['total_aw_shift1'] = total_aw_shift_forward
if len(tmp) > 2:
    num_act_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_act'][:-2]])
    num_act_shift_forward2.index = tmp.index
    tmp['num_act_shift2'] = num_act_shift_forward2
    num_teile_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_teile'][:-2]])
    num_teile_shift_forward2.index = tmp.index
    tmp['num_teile_shift2'] = num_teile_shift_forward2
    total_aw_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['total AW'][:-2]])
    total_aw_shift_forward2.index = tmp.index
    tmp['total_aw_shift2'] = total_aw_shift_forward2
else:
    tmp['num_act_shift2'] = np.NaN
    tmp['num_teile_shift2'] = np.NaN
    tmp['total_aw_shift2'] = np.NaN
# create feature: repair age in year or in month
tmp['first_date'] = tmp.iloc[0, tmp.columns.get_loc('Auftragsdatum')]
tmp['repair_age'] = tmp['Auftragsdatum'] - tmp['first_date']
tmp = tmp.drop('first_date', axis=1)
# create feature: repair frequent in last X month
time_distance_shift1 = pd.concat([pd.Series(np.NaN), tmp['time_distance_from_last'][:-1]])
time_distance_shift1.index = tmp.index
tmp['time_distance_shift1'] = time_distance_shift1
time_distance_shift2 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift1'][:-1]])
time_distance_shift2.index = tmp.index
tmp['time_distance_shift2'] = time_distance_shift2
time_distance_shift3 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift2'][:-1]])
time_distance_shift3.index = tmp.index
tmp['time_distance_shift3'] = time_distance_shift3
tmp['time_distance_from_last2'] = tmp['time_distance_from_last'] + tmp['time_distance_shift1']
tmp['time_distance_from_last3'] = tmp['time_distance_from_last2'] + tmp['time_distance_shift2']
tmp['time_distance_from_last4'] = tmp['time_distance_from_last3'] + tmp['time_distance_shift3']
def get_frequent(x, d = 30):
    if x['time_distance_from_last4'] < timedelta(days = d):
        return 4
    elif x['time_distance_from_last3'] < timedelta(days = d):
        return 3
    elif x['time_distance_from_last2'] < timedelta(days = d):
        return 2
    elif x['time_distance_from_last'] < timedelta(days = d):
        return 1
    else:
        return 0
tmp['frequent_in_last_1_month'] = tmp.apply(get_frequent, axis=1, args = [1*30])
tmp['frequent_in_last_2_month'] = tmp.apply(get_frequent, axis=1, args = [2*30])
tmp['frequent_in_last_3_month'] = tmp.apply(get_frequent, axis=1, args = [3*30])
tmp['frequent_in_last_4_month'] = tmp.apply(get_frequent, axis=1, args = [4*30])
tmp['frequent_in_last_5_month'] = tmp.apply(get_frequent, axis=1, args = [5*30])
tmp['frequent_in_last_6_month'] = tmp.apply(get_frequent, axis=1, args = [6*30])
# create feature: weather come in last X month
tmp['weather_come_in_last_1_month'] = tmp['time_distance_from_last'] < timedelta(days = 1*30)
tmp['weather_come_in_last_2_month'] = tmp['time_distance_from_last'] < timedelta(days = 2*30)
tmp['weather_come_in_last_3_month'] = tmp['time_distance_from_last'] < timedelta(days = 3*30)
tmp['weather_come_in_last_4_month'] = tmp['time_distance_from_last'] < timedelta(days = 4*30)
tmp['weather_come_in_last_5_month'] = tmp['time_distance_from_last'] < timedelta(days = 5*30)
tmp['weather_come_in_last_6_month'] = tmp['time_distance_from_last'] < timedelta(days = 6*30)
# create feature: trend of frequent in the past
def get_trend(x):
    xs = x[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].tolist()
    xs = [x.days for x in xs if not pd.isnull(x)]
    if len(xs) < 2:
        return 0
    ys = range(1, len(xs) + 1)
    xm = sum(xs)/len(xs)
    ym = sum(ys)/len(ys)
    xy = [x*y for x, y in zip(xs, ys)]
    xx = [x*y for x, y in zip(xs, xs)]
    up = sum(xy) - len(xs)*xm*ym
    down = sum(xx) - len(xs)*xm*xm
    if down == 0:
        return 0
    return up*1.0/down
tmp['trend_of_frequent_in_the_past'] = tmp.apply(get_trend, axis=1)
# create feature: weather come in X month again

## extract feature, weather a customer will come in X months for special Gruppe-Nr

In [193]:
tg = df3.copy()

In [194]:
tg['weather_come_in_1_month'] = tg['time_distance_to_next'] < timedelta(days = 1*30)
tg['weather_come_in_2_month'] = tg['time_distance_to_next'] < timedelta(days = 2*30)
tg['weather_come_in_3_month'] = tg['time_distance_to_next'] < timedelta(days = 3*30)
tg['weather_come_in_4_month'] = tg['time_distance_to_next'] < timedelta(days = 4*30)
tg['weather_come_in_5_month'] = tg['time_distance_to_next'] < timedelta(days = 5*30)
tg['weather_come_in_6_month'] = tg['time_distance_to_next'] < timedelta(days = 6*30)

In [195]:
tg

,Auftragsdatum,Fahrgestellnummer,KM-Stand,frequent_in_2014,frequent_in_2015,frequent_in_2016,frequent_in_2017,frequent_in_2018,frequent_in_last_1_month,frequent_in_last_2_month,frequent_in_last_3_month,frequent_in_last_4_month,frequent_in_last_5_month,frequent_in_last_6_month,km_distance_from_last,km_distance_to_next,max_km_distance_in_group,max_time_distance_in_group,mean_km_distance_in_group,mean_time_distance_in_group,min_km_distance_in_group,min_time_distance_in_group,num_act,num_act_shift1,num_act_shift2,num_teile,num_teile_shift1,num_teile_shift2,repair_age,time_distance_from_last,time_distance_from_last2,time_distance_from_last3,time_distance_from_last4,time_distance_shift1,time_distance_shift2,time_distance_shift3,time_distance_to_next,times,total AW,total_aw_shift1,total_aw_shift2,trend_of_frequent_in_the_past,weather_come_in_last_1_month,weather_come_in_last_2_month,weather_come_in_last_3_month,weather_come_in_last_4_month,weather_come_in_last_5_month,weather_come_in_last_6_month,year,weather_come_in_1_month,weather_come_in_2_month,weather_come_in_3_month,weather_come_in_4_month,weather_come_in_5_month,weather_come_in_6_month
0,2014-05-20,------AU-00201251,266265.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,16,NaN,NaN,2,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014,False,False,False,False,False,False
1,2015-08-04,159247535XXXXXXXX,87944.0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,42,NaN,NaN,5,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,6.920609e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2015,False,False,False,False,False,False
2,2014-07-26,1C3ANE9L24X016248,0.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN,NaT,NaN,NaT,3,NaN,NaN,1,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014,False,False,False,False,False,False
3,2014-10-20,1C8FYN8O63T585854,107950.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2192,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,10,NaN,NaN,2,NaN,NaN,0 days,NaT,NaT,NaT,NaT,NaN,NaN,NaN,53 days 00:00:00,8,2.506427e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2014,False,True,True,True,True,True
4,2014-12-12,1C8FYN8O63T585854,110142.0,2,3,2,1,0,0.0,1.0,1.0,1.0,1.0,1.0,2192,636,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,8,10.0,NaN,2,2.0,NaN,53 days,53 days 00:00:00,NaT,NaT,NaT,NaT,NaN,NaN,52 days 00:00:00,8,7.282570e+07,2.506427e+08,NaN,0.000000,False,True,True,True,True,True,2014,False,True,True,True,True,True
5,2015-02-02,1C8FYN8O63T585854,110778.0,2,3,2,1,0,0.0,1.0,1.0,2.0,2.0,2.0,636,587,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,14,8.0,10.0,6,2.0,2.0,105 days,52 days 00:00:00,105 days 00:00:00,NaT,NaT,53 days 00:00:00,NaT,NaN,23 days 00:00:00,8,5.437718e+07,7.282570e+07,2.506427e+08,1.000000,False,True,True,True,True,True,2015,True,True,True,True,True,True
6,2015-02-25,1C8FYN8O63T585854,111365.0,2,3,2,1,0,1.0,1.0,2.0,2.0,3.0,3.0,587,1565,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,4,14.0,8.0,2,6.0,2.0,128 days,23 days 00:00:00,75 days 00:00:00,128 days 00:00:00,NaT,52 days 00:00:00,53 days 00:00:00,NaT,117 days 00:00:00,8,8.299390e+07,5.437718e+07,7.282570e+07,0.051665,True,True,True,True,True,True,2015,False,False,False,True,True,True
7,2015-06-22,1C8FYN8O63T585854,112930.0,2,3,2,1,0,0.0,0.0,0.0,1.0,2.0,2.0,1565,8010,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,5,4.0,14.0,2,2.0,6.0,245 days,117 days 00:00:00,140 days 00:00:00,192 days 00:00:00,245 days 00:00:00,23 days 00:00:00,52 days 00:00:00,53 days 00:00:00,354 days 00:00:00,8,1.740418e+08,8.299390e+07,5.437718e+07,-0.017250,False,False,False,True,True,True,2015,False,False,False,False,False,False
8,2016-06-10,1C8FYN8O63T585854,120940.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,8010,1138,10000.0,365 days,3536.250000,158 days 18:00:00,587.0,23 days,10,5.0,4.0,2

In [196]:
df3 = tg
del tg
gc.collect()

161

## Deal with NA value in df
- Markencode
    - set to unknown
- Adressanredecode
    - set to unknown
- Motorcode
    - set to unknown
- Fahrzeugmodellnummer
    - set to unknown
- Modell
    - set to unknown
- Typ
    - set to unknown
- Getriebecode
    - set to unknown
- Gewicht
    - set to unknown
- Leistung
    - set to unknown
- Erstzulassungsdatum
    - set to unknown
- age_auto
    - set to 0

In [213]:
nd = df.copy()

In [216]:
# fill na with unknown
fs_to_un = ['Markencode', 'Adressanredecode', 'Motorcode', 'Fahrzeugmodellnummer', 'Modell',
           'Typ', 'Getriebecode', 'Gewicht', 'Leistung (KW)', 'Erstzulassungsdatum']
for col in fs_to_un:
    nd[col] = nd[col].fillna('unknown')

In [218]:
nd['age_auto'] = nd['age_auto'].fillna(0)

In [219]:
df = nd
del nd
gc.collect()

239

## Deal with NA value in df3
firstly, drop all the features, which 'times' equal to 1
- frequent_in_last_X_month: 
    - reason: for the first coming auto this feature doesn't exist
    - fill na with 0
- km_distance_from_last:
    - reason: for the first coming auto this feature doesn't exist
    - fill with mean, for the auto that only one time appear, fill with total mean
- km_distance_to_next:
    - reason: doesn't exist for the last data in a group
    - fill with mean
    - Notice: should not be used in the training
- num_act_shift1, num_act_shift2:
    - reason: doesn't exist for the first data in group
    - fill with 0
    - Notice: better not used
- num_teile_shift1, num_teile_shift2:
    - reason: doesn't exist for the first data in group
    - fill with 0
    - Notice: better not used
- repair age:
    - reason: for the first coming auto this feature doesn't exist
    - fill with 0 timedelta
- time_distance_from_last
    - reason: for the first coming auto this feature doesn't exist
    - fill with mean
- time_distance_from_lastX, time_distance_shiftX
    - reason: for the first coming auto this feature doesn't exist
    - fill with mean
    - Notice: better not used
- total_aw_shiftX
    - reason: for the first coming auto this feature doesn't exist
    - fill with 0
    -Notice: better not used

In [197]:
ndf = df3.copy()

In [198]:
# drop times == 1
ndf = ndf.drop(ndf[ndf['times'] == 1].index)

In [199]:
# fill with 0
fs_fill_0 = ['frequent_in_last_1_month', 'frequent_in_last_2_month', 'frequent_in_last_3_month', 
            'frequent_in_last_4_month', 'frequent_in_last_5_month', 'frequent_in_last_6_month',
            'num_act_shift1', 'num_act_shift2', 'num_teile_shift1', 'num_teile_shift2', 'total_aw_shift1',
            'total_aw_shift2']
for col in fs_fill_0:
    ndf[col] = ndf[col].fillna(0)

In [200]:
# fill with 0 timedelta
ndf['repair_age'] = ndf['repair_age'].fillna(timedelta(0))

In [201]:
# fill with mean: km_distance_from_last, km_distance_to_next
ndf['km_distance_from_last'][ndf['km_distance_from_last'].isna()] = ndf['mean_km_distance_in_group'][ndf['km_distance_from_last'].isna()]
ndf['km_distance_from_last'] = ndf['km_distance_from_last'].fillna(ndf['km_distance_from_last'].mean()) # fill the rest with total mean
ndf['km_distance_to_next'][ndf['km_distance_to_next'].isna()] = ndf['mean_km_distance_in_group'][ndf['km_distance_to_next'].isna()]
ndf['km_distance_to_next'] = ndf['km_distance_to_next'].fillna(ndf['km_distance_to_next'].mean())

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [202]:
# fill with mean: time_distance_from_last
# timedelta changed to int automatically, why???
def set_f2_to_f1(x, feature1, feature2):
    #print(x['time_distance_from_last'])
    if type(x[feature1]) == type(pd.NaT):
        x[feature1] = x[feature2]
    return x
ndf = ndf.apply(set_f2_to_f1, axis = 1, args = ['time_distance_from_last', 'mean_time_distance_in_group'])
ndf = ndf.apply(set_f2_to_f1, axis = 1, args = ['time_distance_to_next', 'mean_time_distance_in_group'])
#ndf['time_distance_from_last'][ndf['time_distance_from_last'].isna()] = ndf['mean_time_distance_in_group'][ndf['time_distance_from_last'].isna()]
#ndf['time_distance_from_last'] = ndf['time_distance_from_last'].fillna(ndf['time_distance_from_last'].mean())
#ndf['time_distance_to_next'][ndf['time_distance_to_next'].isna()] = ndf['mean_time_distance_in_group'][ndf['time_distance_to_next'].isna()]
#ndf['time_distance_to_next'] = ndf['time_distance_to_next'].fillna(ndf['time_distance_to_next'].mean())

In [204]:
# fill with mean: time_distance_from_lastX, time_distance_shiftX, total_aw_shiftX


In [189]:
# delete na items in time_distance_from_last


In [205]:
# extract the feature, that we may will used in the training
fs = ['Auftragsdatum', 'Fahrgestellnummer', 'KM-Stand', 'frequent_in_2014', 'frequent_in_2015', 'frequent_in_2016',
     'frequent_in_2017', 'frequent_in_2018', 'frequent_in_last_1_month', 'frequent_in_last_2_month',
     'frequent_in_last_3_month', 'frequent_in_last_4_month', 'frequent_in_last_5_month', 'frequent_in_last_6_month',
     'km_distance_from_last', 'max_km_distance_in_group', 'min_km_distance_in_group', 'mean_km_distance_in_group',
     'mean_time_distance_in_group', 'max_time_distance_in_group', 'min_time_distance_in_group', 'num_act', 
     'num_act_shift1', 'num_act_shift2', 'num_teile', 'num_teile_shift1', 'num_teile_shift2', 'repair_age',
     'time_distance_from_last', 'total AW', 'total_aw_shift1', 'total_aw_shift2', 'trend_of_frequent_in_the_past',
     'weather_come_in_last_1_month', 'weather_come_in_last_2_month', 'weather_come_in_last_3_month',
     'weather_come_in_last_4_month', 'weather_come_in_last_5_month', 'weather_come_in_last_6_month',
     'year', 'weather_come_in_1_month', 'weather_come_in_2_month', 'weather_come_in_3_month', 'weather_come_in_4_month',
     'weather_come_in_5_month', 'weather_come_in_6_month']
ndf = ndf[fs]

In [220]:
# remember that, we still have feature in df, so we have to merge the df3 back to df.
# we name it df2 here for test
df2 = pd.merge(df, ndf, on = ['Fahrgestellnummer', 'Auftragsdatum'], how = 'right')

In [221]:
# length of df2 may larger than length of df3, because the combination of 'Fahrgestellnummer' and 'Auftragsdatum' are
# not unique in df
len(df2)

3019

## Deal with outlier in df
- frequent should be numeric and >= 0
- km distance should be numeric and >= 0
- num_act and num_teile should be numeric and >= 0
- repair_age should be numeric and >= 0
- time distance should be numeric and >= 0

In [222]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3019 entries, 0 to 3018
Data columns (total 68 columns):
Auftragsnummer                   3019 non-null object
num_act_x                        3019 non-null int64
total AW_x                       3019 non-null float64
Gruppe-Nr                        3019 non-null object
num_teile_x                      3019 non-null int64
KM-Stand_x                       3019 non-null float64
Markencode                       3019 non-null object
Lagerortcode                     3019 non-null object
Auftragsdatum                    3019 non-null datetime64[ns]
Adressanredecode                 3019 non-null object
Motorcode                        3019 non-null object
Fahrzeugmodellnummer             3019 non-null object
Modell                           3019 non-null object
Typ                              3019 non-null object
Getriebecode                     3019 non-null object
Gewicht                          3019 non-null object
Leistung (KW)        

## extract feature: mean_km_distance_in_total, mean_time_distance_in_total

In [224]:
tt = df3.copy()
tt['mean_km_distance_in_total'] = tt['km_distance_from_last'].mean()
#tt['mean_time_distance_in_total'] = tt['time_distance_from_last'].mean()

## feature clustering

## encoding and normalization

## deal with data balance

## distribute and save for training

In [25]:
def check_to_1(df, atta, attb):
    """
    input:
        df, Dataframe:
            the data frame
        atta, string:
            first attribute
        attb, string:
            second attribute
    output:
        out boolean:
            if for an a has more than one b, return True, otherwise return False
    """
    df = df[[atta, attb]]
    df.to_csv('/tmp/tmp.csv', sep = ';', index = False)
    abdict = ABDict('/tmp/tmp.csv', header = True, sep = ';')
    dic = abdict.dic
    flag = 0
    for i,j in dic.items():
        if len(j) > 1:
            print(str(i) + '\t' + str(j))

In [1]:
# 转化为auftrag table， 但是这次合并的是Teile-Nr项
# 给的数据的每一行都是一个维修项，初衷是，把属于同一个auftrag的维修项合并到一起，看一下，在同一个Auftrag中，经常一起修的是那些内容

def toAuftragTable(df, att, auftn, clean = True):
    """
    input:
        df, DataFrame:
            the dataframe
        att, string:
            the column name of the target attribute
        auftn, string:
            the column name of the aftragsnummer attribute
        clean:
            when true, drop the null item in auftn attribute.
    output:
        df_g, DataFrame:
            dataframe contrains two columns auftn and att
            type of item in att is string, separate with ';'
    """
    # assert: make sure the type of the attributes inputted
    
    # extract the att and date columns
    df = df[[att, auftn]]
    # set type to object
    #df[att] = df[att].astype('object')
    #df[auftn] = df[auftn].astype('object')
    # if clean is True, drop the fake data, like the null data
    if clean:
        print("Falls Null date exist, drop these dates directly")
        #df = df.drop(df[df[att].isnull()].index)
        df = df.drop(df[df[auftn].isnull()].index)
    # group and sum 
    df_g = df.groupby([auftn], as_index = False).apply(agg)
    return df_g

# apply 只能对单行进行处理，而不是对整个分组进行处理，所以估计应该把axis换成1，比较好
def agg(x):
    # 是否用‘ ’分隔会比较好，这样就不用对初始的属性，
    # x 在这里是dataframe？？？
    #x = [str(i) for i in x]
    x = x.apply(lambda x: ';'.join(set([str(i) for i in x])), axis = 0)
    #x = x.apply(lambda x: ' '.join(set(x)), axis = 0)
    #print(x.columns.values)
    return x

In [2]:
class ABDict:
    """
    make sure that only two columns in the df, otherwise only the first two columns will be used
    用于两列数据互补空值
    """
    def __init__(self, path, header = True, sep = ';'):
        self.dic = self._load_dict(path, header, sep) 
        self.dic_conv = self._load_dict_conv(path, header, sep)
        print(len(self.dic))
        print(len(self.dic_conv))
        counter = 0
        ########
        # test #
        ########
        #for i in self.dic.keys():
        #    if counter > 1:
        #        break
        #    print(i, self.dic[i])
        #    counter += 1
        #print('schraube' in self.dic_conv.keys())
    
    def AToB(self, a):
        if a in self.dic.keys():
            out = self.dic[a][0]
            if type(out) == list:
                print(out)
            return out
        else:
            #print(a)
            return float('nan')
    
    def BToA(self, b):
        if b in self.dic_conv.keys():
            out = self.dic_conv[b][0]
            #print(out)
            return out
        else:
            #print(b)
            return float('nan')

    def _load_dict(self, path, header, sep):
        dic = {} 
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter ==0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()   # 忘记去换行符了，导致调了一晚上的错
                items = li.split(sep)
                if items[0] not in dic.keys():
                    # if not exit, add new item
                    if items[1] != 'nan':
                        dic[items[0]] = [items[1]]
                elif items[1] not in dic[items[0]]:
                    # if item exit but value not eixt, add the value to the list
                    dic[items[0]].append(items[1])
                counter += 1
        return dic
    
    def _load_dict_conv(self, path, header, sep):
        dic = {}
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter == 0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()
                items = li.split(sep)
                if items[1] not in dic.keys():
                    if items[0] != 'nan':
                        #print(items[0])
                        dic[items[1]] = [items[0]]
                elif items[0] not in dic[items[1]]:
                    dic[items[1]].append(items[0])
                counter += 1
        return dic